In [58]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
from collections import Counter

## Compute mean, var, num of ratings for posts (obsolete)

In [7]:
posts = pickle.load(open('data/posts_with_age_gender.dat', 'rb'))
comments = pickle.load(open('data/comments_with_ratings_df.dat', 'rb'))

In [20]:
def noname():
    global ratings, posts
    for i, (index, row) in tqdm(enumerate(posts.iterrows())):
        ratings = comments[comments.parent_post_id == row.id].rating.values
        ratings = ratings[~np.isnan(ratings)]
        posts.set_value(index, 'ratings_mean', ratings.mean())
        posts.set_value(index, 'ratings_var', ratings.var())
        posts.set_value(index, 'ratings_num', ratings.shape[0])
        if i > 20:
            break

In [18]:
%load_ext line_profiler

In [19]:
%lprun -f noname noname()

In [4]:
for index, row in tqdm(posts.iterrows()):
    ratings = comments[comments.parent_post_id == row.id].rating.values
    ratings = ratings[~np.isnan(ratings)]
    posts.set_value(index, 'ratings_mean', ratings.mean())
    posts.set_value(index, 'ratings_var', ratings.var())
    posts.set_value(index, 'ratings_num', ratings.shape[0])

0it [00:00, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: Degrees of freedom <= 0 for slice
  """
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
28686it [17:58, 22.54it/s]


In [7]:
pickle.dump(posts, open('data/posts_with_ratings_summary.dat', 'wb'))

## Getting post ids of comments

In [ ]:
comments = pickle.load(open('data/comments_with_ratings_df.dat', 'rb'))

In [ ]:
# failures = 0
for i, (index, comment) in tqdm(enumerate(comments.iterrows())):
    try:  # to find root of the current comment tree and take its parent_post_id
        cur_comment = comment
        while cur_comment.parent_post_id is None:
#             print(cur_comment.parent_comment_id)
            cur_comment = comments[comments.id == cur_comment.parent_comment_id].iloc[0]
        comments.set_value(index, 'parent_post_id', cur_comment.parent_post_id)
    except:
        pass
#         failures += 1
#     if failures % 500 == 0:
#         print(str(failures) + '/' + str(i) + ' failures')

In [24]:
comments = comments[comments.parent_post_id.notnull()]

In [25]:
pickle.dump(comments, open('data/comments_with_post_ids.dat', 'wb'))

## pd.merge ratings and posts

In [26]:
comments = pickle.load(open('data/comments_with_post_ids.dat', 'rb'))

In [27]:
comments

,body,created,edited,gilded,id,is_submitter,name,parent_comment_id,parent_post_id,permalink,replies,score,body_len,rating
created,,,,,,,,,,,,,,
2009-12-15 06:10:51,Shorter hair. for sure. No facial hair! Beau...,2009-12-15 06:10:51,0.0,False,c0h7wut,False,emmadilemma,None,aes21,/r/Rateme/comments/aes21/nervous_but_ill_give_...,"[c0h7wwz, c0h7x0z, c0mohob, c0h7x2y, c0h7x78, ...",1,57,NaN
2009-12-15 06:12:06,Thank you. :) How short do you think I should ...,2009-12-15 06:12:06,0.0,False,c0h7wwz,True,djantigravity,c0h7wut,aes21,/r/Rateme/comments/aes21/nervous_but_ill_give_...,"[c0h7x0z, c0mohob, c0h7x2y, c0h7x78, c0h7x95]",1,53,NaN
2009-12-15 06:14:20,don't go too short. look at chace crawford. ...,2009-12-15 06:14:20,0.0,False,c0h7x0z,False,emmadilemma,c0h7wwz,aes21,/r/Rateme/comments/aes21/nervous_but_ill_give_...,"[c0h7x2y, c0h7x78, c0h7x95]",1,105,NaN
2010-03-22 17:41:07,Don't listen to her. you need hair. Don't over...,2010-03-22 17:41:07,0.0,False,c0mohob,False,ZenaLundgren,c0h7wwz,aes21,/r/Rateme/comments/aes21/nervous_but_ill_give_...,[],1,344,NaN
2009-12-15 06:15:14,So have the hair stop right above the eyebrows?,2009-12-15 06:15:14,0.0,False,c0h7x2y,True,djantigravity,c0h7x0z,aes21,/r/Rateme/comments/aes21/nervous_but_ill_give_...,"[c0h7x78, c0h7x95]",1,47,NaN
2009-12-15 06:17:29,yes. and keep it tidy. think gel. or waterw...,2009-12-15 06:17:29,0.0,False,c0h7x78,False,emmadilemma,c0h7x2y,aes21,/r/Rateme/comments/aes21/nervous_but_ill_give_...,[c0h7x95],1,83,NaN
2009-12-15 06:18:38,All right. Thank you :). That has been the big...,2009-12-15 06:18:38,0.0,False,c0h7x95,True,djantigravity,c0h7x78,aes21,/r/Rateme/comments/aes21/nervous_but_ill_give_...,[],1,106,NaN
2009-12-15 06:02:00,that's some serious growing. you did there!\n\...,2009-12-15 06:02:00,0.0,False,c0h7wer,False,emmadilemma,None,aeryj,/r/Rateme/comments/aeryj/i_posted_pictures_in_...,[],6,138,NaN
2010-02-18 03:21:18,Big guy. Good potential.,2010-02-18 03:21:18,0.0,False,c0kr76z,False,SpankmasterS,None,aeryj,/r/Rateme/comments/aeryj/i_posted_pictures_in_...,[],2,25,NaN


In [28]:
posts_renamed = posts.rename(columns={col: 'post_' + col for col in posts.columns})
comments_renamed = comments.rename(columns={col: 'comment_' + col for col in comments.columns})

In [36]:
merged = posts_renamed.merge(comments_renamed, left_on='post_id',
                             right_on='comment_parent_post_id')

In [64]:
# pd.set_option("display.max_columns", 50)

In [66]:
merged = merged.drop(['comment_parent_post_id', 'comment_edited'], axis='columns')

In [68]:
merged = merged.rename(columns={'comment_name': 'comment_author'})

In [98]:
merged = merged[['post_gender', 'post_age', 'post_score', 'comment_rating', 'comment_score', 'post_author', 'comment_author',
        'post_title', 'comment_body', 'post_selftext',
        'post_created', 'comment_created', 'post_author_has_flair', 'post_edited', 'post_gilded',
        'post_height', 'post_width',
        'post_thumbnail_height', 'post_thumbnail_width',
        'post_is_self', 'post_over_18', 'comment_gilded', 'comment_is_submitter',
        'comment_body_len',
        'post_id', 'comment_id', 'comment_parent_comment_id', 'comment_replies',
        'post_permalink', 'comment_permalink']]

In [ ]:
comment_author_counts = merged.comment_author.value_counts()
data['comment_author_num_comments'] = merged.comment_author.apply(lambda x: comment_author_counts.loc[x])

In [101]:
pickle.dump(merged, open('data/merged.dat', 'wb'))